
# Florida Case Study — Hurricane Irma (2017): Forecasting Agent Notebook



In [ ]:

#@title Install & Imports (Colab/Local)
# If running locally and these are already installed, you may skip.
!pip -q install pandas numpy matplotlib scikit-learn torch torchvision torchaudio statsmodels requests pytz --upgrade

import os
import io
import math
import json
import time
import pytz
import zipfile
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime, timedelta, timezone
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.arima.model import ARIMA

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

plt.rcParams["figure.figsize"] = (7,5)



## 1) Data Ingestion

We try two sources (either should work; use whichever responds).  
- **IBTrACS v4** CSV (global archive) — filter to *IRMA (2017)* in the North Atlantic.  
- **NHC HURDAT2** Best Track text — parse AL112017 (Irma).

> If one URL fails due to maintenance, run the other cell or paste a working mirror.


In [ ]:

#@title Define helper: robust HTTP GET (with retries)
def http_get(url, max_tries=3, timeout=60):
    last_e = None
    for i in range(max_tries):
        try:
            r = requests.get(url, timeout=timeout)
            if r.status_code == 200:
                return r
            last_e = RuntimeError(f"HTTP {r.status_code} for {url}")
        except Exception as e:
            last_e = e
        time.sleep(2*(i+1))
    if last_e:
        raise last_e


In [ ]:

#@title Option A: IBTrACS CSV (Filter for Irma 2017 in North Atlantic)
# IBTrACS v04 CSV (global list). If this path changes, search "ibtracs v04 csv ALL list" and update.
IBTRACS_URL = "https://www.ncei.noaa.gov/data/international-best-track-archive-for-climate-stewardship-ibtracs/v04r00/provisional/csv/ibtracs.ALL.list.v04r00.csv"

try:
    r = http_get(IBTRACS_URL)
    df = pd.read_csv(io.StringIO(r.text))
    # Common IBTrACS columns (may vary slightly by release):
    # 'NAME', 'SEASON', 'BASIN', 'ISO_TIME', 'LAT', 'LON', 'USA_SSHS', 'USA_WIND', 'USA_PRES', etc.
    # Filter Irma 2017 in North Atlantic (NA)
    df_irma = df[
        (df['NAME'].str.upper() == 'IRMA') & 
        (df['SEASON'] == 2017) & 
        (df['BASIN'].str.upper().str.contains('NA'))
    ].copy()

    # Fallback if 'BASIN' label differs
    if df_irma.empty and 'SUBBASIN' in df.columns:
        df_irma = df[
            (df['NAME'].str.upper() == 'IRMA') & 
            (df['SEASON'] == 2017) & 
            (df['SUBBASIN'].str.upper().str.contains('NA'))
        ].copy()

    if df_irma.empty:
        raise ValueError("Could not filter Irma 2017 in NA from IBTrACS. Try the HURDAT2 option below.")

    # Clean & coerce
    # Normalize time
    df_irma['ISO_TIME'] = pd.to_datetime(df_irma['ISO_TIME'], errors='coerce', utc=True)
    # Use USA_WIND and USA_PRES if available, else WIND/PRES
    wind_col = 'USA_WIND' if 'USA_WIND' in df_irma.columns else 'WIND'
    pres_col = 'USA_PRES' if 'USA_PRES' in df_irma.columns else 'PRES'

    df_irma = df_irma[['ISO_TIME','LAT','LON',wind_col, pres_col]].rename(
        columns={wind_col:'WIND_KT', pres_col:'PRES_MB'}
    )
    df_irma = df_irma.dropna(subset=['ISO_TIME','LAT','LON'])
    df_irma = df_irma.sort_values('ISO_TIME').reset_index(drop=True)

    source_used = "IBTrACS"
    print(f"Loaded {len(df_irma)} Irma records from IBTrACS.")
except Exception as e:
    print("IBTrACS failed with:", e)
    df_irma = None
    source_used = None


In [ ]:

#@title Option B: NHC HURDAT2 (parse AL112017 Irma)
# If IBTrACS failed above, try HURDAT2.
# This URL may be updated yearly; if it 404s, search "hurdat2 atlantic text file NHC" and update.
HURDAT_URL = "https://www.nhc.noaa.gov/data/hurdat/hurdat2-1851-2023-120523.txt"

if df_irma is None:
    try:
        txt = http_get(HURDAT_URL).text
        # Find AL112017 — Irma block
        lines = txt.splitlines()
        start_idx = None
        for i, line in enumerate(lines):
            if "AL, 11, 2017" in line and "IRMA" in line.upper():
                start_idx = i
                break
        if start_idx is None:
            raise ValueError("Could not find AL112017 Irma header in HURDAT2 file.")

        # Header line format like:
        # AL, 11, 2017, IRMA, ... number of entries
        header = lines[start_idx]
        # next N lines are the best-track entries; count comes after header commas
        # e.g., "AL, 11, 2017, IRMA, 137, ..."
        parts = [p.strip() for p in header.split(",")]
        n_entries = int(parts[4])

        rows = []
        for j in range(1, n_entries+1):
            L = lines[start_idx + j]
            # Format documented by NHC: date,time,record id,status,lat,lon,max wind,min pres,...
            # Example: 20170901, 0000, , TD,  17.0N,  33.0W,  25, 1007, ...
            vals = [v.strip() for v in L.split(",")]
            dts = datetime.strptime(vals[0] + vals[1], "%Y%m%d%H%M").replace(tzinfo=timezone.utc)
            lat = float(vals[4][:-1]) * (1 if vals[4].endswith("N") else -1)
            lon = float(vals[5][:-1]) * (-1 if vals[5].endswith("W") else 1)
            wind = float(vals[6]) if vals[6] != "" else np.nan
            pres = float(vals[7]) if vals[7] != "" else np.nan
            rows.append((dts, lat, lon, wind, pres))
        df_irma = pd.DataFrame(rows, columns=["ISO_TIME","LAT","LON","WIND_KT","PRES_MB"]).sort_values("ISO_TIME").reset_index(drop=True)
        source_used = "HURDAT2"
        print(f"Loaded {len(df_irma)} Irma records from HURDAT2.")
    except Exception as e:
        print("HURDAT2 failed with:", e)

if df_irma is None:
    raise SystemExit("No data loaded from either source. Please update the URLs and rerun.")



## 2) Preprocess & Feature Engineering

We align timestamps, compute deltas (speed/bearing proxies), and build features for short‑term forecasting:
- Inputs `X`: 

  `LAT_t, LON_t, WIND_t, PRES_t, ΔLAT_t, ΔLON_t, hour_sin, hour_cos, day_sin, day_cos`  
- Targets `y`: next‑step `(LAT_{t+1}, LON_{t+1}, WIND_{t+1})`


In [ ]:

df = df_irma.copy()
df = df.dropna(subset=["LAT","LON"]).copy()
df["WIND_KT"] = df["WIND_KT"].astype(float)
df["PRES_MB"] = df["PRES_MB"].astype(float)

# Ensure uniform 3-hour steps (many best-track datasets are 6-hour; we can forward-fill if needed)
df = df.set_index("ISO_TIME").sort_index()
df = df.resample("3H").interpolate(limit_direction="both")
df = df.reset_index()

# Deltas (approx)
df["DLAT"] = df["LAT"].diff().fillna(0.0)
df["DLON"] = df["LON"].diff().fillna(0.0)

# Cyclical time features
t = df["ISO_TIME"].dt.tz_convert("UTC") if df["ISO_TIME"].dt.tz is not None else df["ISO_TIME"].dt.tz_localize("UTC")
hour = t.dt.hour.values
day_of_year = t.dt.dayofyear.values

df["HOUR_SIN"] = np.sin(2*np.pi*hour/24.0)
df["HOUR_COS"] = np.cos(2*np.pi*hour/24.0)
df["DOY_SIN"]  = np.sin(2*np.pi*day_of_year/365.0)
df["DOY_COS"]  = np.cos(2*np.pi*day_of_year/365.0)

# Targets: next-step
df["LAT_next"]  = df["LAT"].shift(-1)
df["LON_next"]  = df["LON"].shift(-1)
df["WIND_next"] = df["WIND_KT"].shift(-1)

df = df.dropna().reset_index(drop=True)
print(df.head())



## 3) Train/Test Split & Dataset Loader

We use the first 80% for training and last 20% for testing (time-ordered).  
Sequence length can be tuned via `SEQ_LEN`.


In [ ]:

FEATURES = ["LAT","LON","WIND_KT","PRES_MB","DLAT","DLON","HOUR_SIN","HOUR_COS","DOY_SIN","DOY_COS"]
TARGETS  = ["LAT_next","LON_next","WIND_next"]

all_X = df[FEATURES].values.astype(np.float32)
all_y = df[TARGETS].values.astype(np.float32)

n = len(df)
n_train = int(0.8 * n)
train_X, test_X = all_X[:n_train], all_X[n_train:]
train_y, test_y = all_y[:n_train], all_y[n_train:]

# Scale features (targets in native units for interpretability)
scaler = StandardScaler()
train_X_sc = scaler.fit_transform(train_X)
test_X_sc  = scaler.transform(test_X)

SEQ_LEN = 6  # use past 6 steps (~18 hours with 3h interval)

def make_sequences(X, y, seq_len):
    Xs, ys = [], []
    for i in range(len(X)-seq_len):
        Xs.append(X[i:i+seq_len])
        ys.append(y[i+seq_len])
    return np.stack(Xs), np.stack(ys)

Xtr_seq, ytr_seq = make_sequences(train_X_sc, train_y, SEQ_LEN)
Xte_seq, yte_seq = make_sequences(test_X_sc,  test_y,  SEQ_LEN)

print("Train seq:", Xtr_seq.shape, "Test seq:", Xte_seq.shape)



## 4) LSTM Forecasting Model

A small, fast **LSTM** predicts next‑step `(lat, lon, wind)` from the last `SEQ_LEN` windows of features.


In [ ]:

class LSTMForecast(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=1, output_dim=3, dropout=0.1):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True, dropout=dropout if num_layers>1 else 0.0)
        self.fc   = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # x: (B, T, F)
        out, _ = self.lstm(x)
        last = out[:, -1, :]  # (B, H)
        return self.fc(last)

class SeqDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_ds = SeqDataset(Xtr_seq, ytr_seq)
test_ds  = SeqDataset(Xte_seq, yte_seq)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=32, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMForecast(input_dim=Xtr_seq.shape[-1], hidden_dim=96, num_layers=1, output_dim=3).to(device)
criterion = nn.L1Loss()  # MAE
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


In [ ]:

#@title Train the model
EPOCHS = 50
for epoch in range(1, EPOCHS+1):
    model.train()
    loss_sum = 0.0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        pred = model(xb)
        loss = criterion(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_sum += loss.item() * len(xb)
    train_loss = loss_sum / len(train_ds)

    # quick val
    model.eval()
    with torch.no_grad():
        val_sum = 0.0
        for xb, yb in test_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            loss = criterion(pred, yb)
            val_sum += loss.item() * len(xb)
        val_loss = val_sum / len(test_ds)

    if epoch % 10 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d} | train MAE {train_loss:.4f} | val MAE {val_loss:.4f}")



## 5) Evaluation vs Persistence Baseline

- **Model**: next‑step MAE for latitude, longitude, and wind.  
- **Baseline (Persistence)**: predict next = current (`LAT_t → LAT_{t+1}`, etc.).


In [ ]:

model.eval()
with torch.no_grad():
    pred_test = []
    for xb, yb in test_loader:
        xb = xb.to(device)
        pred = model(xb).cpu().numpy()
        pred_test.append(pred)
    pred_test = np.vstack(pred_test)

# Align to original unwindowed test targets:
y_true = yte_seq

mae_lat = mean_absolute_error(y_true[:,0], pred_test[:,0])
mae_lon = mean_absolute_error(y_true[:,1], pred_test[:,1])
mae_wnd = mean_absolute_error(y_true[:,2], pred_test[:,2])

print(f"Model MAE — LAT: {mae_lat:.3f}°, LON: {mae_lon:.3f}°, WIND: {mae_wnd:.3f} kt")

# Persistence baseline on the same alignment: use the last step in the input sequence
persist = Xte_seq[:, -1, :]
# map indices in FEATURES to lat/lon/wind positions
i_lat = FEATURES.index("LAT")
i_lon = FEATURES.index("LON")
i_wnd = FEATURES.index("WIND_KT")

persist_lat = persist[:, i_lat]
persist_lon = persist[:, i_lon]
persist_wnd = persist[:, i_wnd]

b_mae_lat = mean_absolute_error(y_true[:,0], persist_lat)
b_mae_lon = mean_absolute_error(y_true[:,1], persist_lon)
b_mae_wnd = mean_absolute_error(y_true[:,2], persist_wnd)

print(f"Baseline (Persistence) MAE — LAT: {b_mae_lat:.3f}°, LON: {b_mae_lon:.3f}°, WIND: {b_mae_wnd:.3f} kt")



## 6) Visualize Track & Wind Forecasts

We plot **actual vs predicted** for the test window (time‑aligned to predictions).


In [ ]:

# Build a time index matching predictions in test set
test_times = df["ISO_TIME"].iloc[len(df) - len(test_X) + SEQ_LEN : len(df)]  # windowed alignment
test_times = test_times.reset_index(drop=True)[:len(pred_test)]

fig = plt.figure()
plt.plot(y_true[:,1], y_true[:,0], marker='o', linewidth=1, label="Actual Track")
plt.plot(pred_test[:,1], pred_test[:,0], marker='x', linewidth=1, label="Predicted Track")
plt.xlabel("Longitude (°)")
plt.ylabel("Latitude (°)")
plt.title("Hurricane Irma — Track (Test Period)")
plt.legend()
plt.show()

fig = plt.figure()
plt.plot(test_times, y_true[:,2], label="Actual Wind (kt)")
plt.plot(test_times, pred_test[:,2], label="Predicted Wind (kt)")
plt.xlabel("Time (UTC)")
plt.ylabel("Max Sustained Wind (kt)")
plt.title("Hurricane Irma — Wind Forecast (Test Period)")
plt.legend()
plt.xticks(rotation=20)
plt.tight_layout()
plt.show()



## 7) (Optional) ARIMA Baseline for Wind Only

A simple univariate ARIMA on wind (best‑track series) for comparison.


In [ ]:

# Fit ARIMA(p,d,q) on training wind; forecast next steps to align with test length
train_wind = train_y[:,2]  # WIND_next
# We'll model the current wind (one‑step shifted): align with simplicity
wind_series = df["WIND_KT"].values.astype(float)
split_idx = len(train_X)  # train/test split in raw index
wind_train = wind_series[:split_idx]
wind_test  = wind_series[split_idx:]

# small ARIMA; if it fails to converge, tweak order
try:
    arima = ARIMA(wind_train, order=(2,1,2)).fit()
    fc = arima.forecast(steps=len(wind_test))
    mae_arima = mean_absolute_error(wind_test, fc)
    print(f"ARIMA Wind-only MAE (kt): {mae_arima:.3f}")
except Exception as e:
    print("ARIMA failed:", e)



## 8) Save/Load Trained Model

Save a minimal checkpoint (state_dict + feature list + scaler stats) for deployment.


In [ ]:

CKPT_PATH = "irma_lstm_checkpoint.pt"
meta = {
    "features": FEATURES,
    "seq_len": SEQ_LEN,
    "scaler_mean": scaler.mean_.tolist(),
    "scaler_scale": scaler.scale_.tolist(),
    "source_used": source_used,
}
torch.save({
    "state_dict": model.state_dict(),
    "meta": meta,
}, CKPT_PATH)

print("Saved:", CKPT_PATH, "| source:", source_used)



## 9) (Optional) FastAPI Serving Stub

Use this snippet in a separate `server.py` to expose a `/predict` endpoint.  
It expects the **last `SEQ_LEN` frames of features** matching `FEATURES`.


In [ ]:

api_stub = r'''
from fastapi import FastAPI
import uvicorn
import torch
import torch.nn as nn
import numpy as np
import json

from pydantic import BaseModel

FEATURES = {features}
SEQ_LEN = {seq_len}

class LSTMForecast(nn.Module):
    def __init__(self, input_dim, hidden_dim=96, num_layers=1, output_dim=3, dropout=0.1):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True, dropout=dropout if num_layers>1 else 0.0)
        self.fc   = nn.Linear(hidden_dim, output_dim)
    def forward(self, x):
        out, _ = self.lstm(x)
        last = out[:, -1, :]
        return self.fc(last)

class Window(BaseModel):
    x: list  # shape = [SEQ_LEN, len(FEATURES)]

def load_model(ckpt_path="irma_lstm_checkpoint.pt"):
    ckpt = torch.load(ckpt_path, map_location="cpu")
    meta = ckpt["meta"]
    model = LSTMForecast(input_dim=len(meta["features"]))
    model.load_state_dict(ckpt["state_dict"])
    model.eval()
    return model, meta

app = FastAPI()
model, meta = load_model()

@app.post("/predict")
def predict(window: Window):
    X = np.array(window.x, dtype=np.float32)
    if X.shape != (SEQ_LEN, len(FEATURES)):
        return {"error": f"Expected shape ({SEQ_LEN}, {len(FEATURES)})"}
    # Standardize using saved scaler stats
    mean = np.array(meta["scaler_mean"], dtype=np.float32)
    scale = np.array(meta["scaler_scale"], dtype=np.float32)
    Xs = (X - mean) / scale
    xb = torch.tensor(Xs, dtype=torch.float32).unsqueeze(0)  # (1, T, F)
    with torch.no_grad():
        pred = model(xb).numpy().tolist()[0]
    return {"lat_next": pred[0], "lon_next": pred[1], "wind_next_kt": pred[2]}

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)
'''.format(features=json.dumps(FEATURES), seq_len=SEQ_LEN)

with open("fastapi_stub_server.py", "w") as f:
    f.write(api_stub)

print("Wrote fastapi_stub_server.py")
print("\nRun locally:")
print("  uvicorn fastapi_stub_server:app --reload --port 8000")
